In [2]:
!nvidia-smi

Wed Jan  7 05:40:35 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   31C    P0             51W /  400W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [3]:
!git clone https://github.com/PerforatedAI/PerforatedAI.git
%cd PerforatedAI

# install
!pip install -e .

# Install dependencies
!pip install -q transformers datasets accelerate wandb

Cloning into 'PerforatedAI'...
remote: Enumerating objects: 1383, done.
remote: Counting objects: 100% (339/339), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 1383 (delta 249), reused 230 (delta 230), pack-reused 1044 (from 2)
Receiving objects: 100% (1383/1383), 72.27 MiB | 39.34 MiB/s, done.
Resolving deltas: 100% (764/764), done.
/content/PerforatedAI
Obtaining file:///content/PerforatedAI
  Preparing metadata (setup.py) ... done
  Running setup.py develop for perforatedai


In [4]:
import os
import math
import random
import numpy as np

import torch
from torch.utils.data import DataLoader

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    get_linear_schedule_with_warmup,
)

from perforatedai import globals_perforatedai as GPA
from perforatedai import utils_perforatedai as UPA

import wandb

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Building dendrites without Perforated Backpropagation
Using device: cuda


In [5]:
# Models
BASELINE_MODEL_NAME = "distilbert-base-uncased"      # baseline
COMPRESSED_MODEL_NAME = "prajjwal1/bert-tiny"        # smaller "compressed" model

DATASET_NAME = "glue"
DATASET_CONFIG = "sst2"  # sentiment classification
NUM_LABELS = 2

BATCH_SIZE = 32
BASELINE_EPOCHS = 3
COMPRESSED_EPOCHS = 3
INITIAL_LR = 2e-5

SEED = 42

# W&B config (optional but recommended)
WANDB_PROJECT = "perforatedai-hackathon"
os.environ["WANDB_PROJECT"] = WANDB_PROJECT

In [6]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(SEED)

In [7]:
import wandb
wandb.login(key="4d4d4c8f520ba4f560db3da3a2e051523a812c46")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: warkevishal29 (warkevishal292) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [8]:
tokenizer = AutoTokenizer.from_pretrained(BASELINE_MODEL_NAME)

raw_datasets = load_dataset(DATASET_NAME, DATASET_CONFIG)

def preprocess_function(examples):
    return tokenizer(
        examples["sentence"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )

encoded_datasets = raw_datasets.map(preprocess_function, batched=True)

encoded_datasets = encoded_datasets.rename_column("label", "labels")
encoded_datasets.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"],
)

train_dataset = encoded_datasets["train"]
eval_dataset = encoded_datasets["validation"]

print(train_dataset[0])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

{'labels': tensor(0), 'input_ids': tensor([  101,  5342,  2047,  3595,  8496,  2013,  1996, 18643,  3197,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0, 

In [9]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
eval_dataloader = DataLoader(eval_dataset, batch_size=BATCH_SIZE)

In [10]:
def evaluate(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    loss_sum = 0.0

    with torch.no_grad():
        for batch in dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            logits = outputs.logits

            preds = torch.argmax(logits, dim=-1)
            correct += (preds == batch["labels"]).sum().item()
            total += batch["labels"].size(0)
            loss_sum += loss.item() * batch["labels"].size(0)

    avg_loss = loss_sum / total
    accuracy = correct / total
    return {"loss": avg_loss, "accuracy": accuracy}

In [11]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def benchmark_inference(model, dataloader, n_batches=10):
    model.eval()
    import time
    start = time.time()
    with torch.no_grad():
        for i, batch in enumerate(dataloader):
            if i >= n_batches:
                break
            batch = {k: v.to(device) for k, v in batch.items()}
            _ = model(**batch)
    end = time.time()
    return (end - start) / n_batches

In [12]:
baseline_model = AutoModelForSequenceClassification.from_pretrained(
    BASELINE_MODEL_NAME,
    num_labels=NUM_LABELS,
).to(device)

baseline_optimizer = torch.optim.AdamW(baseline_model.parameters(), lr=INITIAL_LR)

num_training_steps = BASELINE_EPOCHS * math.ceil(len(train_dataloader))
baseline_scheduler = get_linear_schedule_with_warmup(
    baseline_optimizer,
    num_warmup_steps=int(0.1 * num_training_steps),
    num_training_steps=num_training_steps,
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
def train_baseline(model, optimizer, scheduler, num_epochs=3, run_name="baseline-no-dendrites"):
    run = wandb.init(project=WANDB_PROJECT, name=run_name)
    global_step = 0

    for epoch in range(num_epochs):
        model.train()
        for batch in train_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

            global_step += 1
            if global_step % 50 == 0:
                wandb.log({"train/loss": loss.item(), "step": global_step})

        metrics = evaluate(model, eval_dataloader)
        print(f"[Baseline] Epoch {epoch} | val_loss={metrics['loss']:.4f} | val_acc={metrics['accuracy']:.4f}")
        wandb.log(
            {
                "val/loss": metrics["loss"],
                "val/accuracy": metrics["accuracy"],
                "epoch": epoch,
            }
        )

    run.finish()
    return model

baseline_model = train_baseline(baseline_model, baseline_optimizer, baseline_scheduler, BASELINE_EPOCHS)
baseline_metrics = evaluate(baseline_model, eval_dataloader)
print("Baseline metrics:", baseline_metrics)
print("Baseline params:", count_parameters(baseline_model))

[Baseline] Epoch 0 | val_loss=0.2427 | val_acc=0.9048
[Baseline] Epoch 1 | val_loss=0.2957 | val_acc=0.9048
[Baseline] Epoch 2 | val_loss=0.3570 | val_acc=0.9025


epoch,▁▅█
step,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇████
train/loss,█▅▅▅▄▂▅▄▂▃▂▃▃▃▂▁▃▂▁▁▂▁▄▂▃▁▂▁▁▁▃▁▁▂▁▄▂▃▁▁
val/accuracy,██▁
val/loss,▁▄█
epoch,2
step,6300
train/loss,0.03724
val/accuracy,0.90252
val/loss,0.35695


Baseline metrics: {'loss': 0.3569536072740314, 'accuracy': 0.9025229357798165}
Baseline params: 66955010


In [14]:
compressed_model = AutoModelForSequenceClassification.from_pretrained(
    COMPRESSED_MODEL_NAME,
    num_labels=NUM_LABELS,
).to(device)

compressed_optimizer = torch.optim.AdamW(compressed_model.parameters(), lr=INITIAL_LR)

num_training_steps = COMPRESSED_EPOCHS * math.ceil(len(train_dataloader))
compressed_scheduler = get_linear_schedule_with_warmup(
    compressed_optimizer,
    num_warmup_steps=int(0.1 * num_training_steps),
    num_training_steps=num_training_steps,
)

config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
def train_compressed(model, optimizer, scheduler, num_epochs=3, run_name="compressed-no-dendrites"):
    run = wandb.init(project=WANDB_PROJECT, name=run_name)
    global_step = 0

    for epoch in range(num_epochs):
        model.train()
        for batch in train_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

            global_step += 1
            if global_step % 50 == 0:
                wandb.log({"train/loss": loss.item(), "step": global_step})

        metrics = evaluate(model, eval_dataloader)
        print(f"[Compressed] Epoch {epoch} | val_loss={metrics['loss']:.4f} | val_acc={metrics['accuracy']:.4f}")
        wandb.log(
            {
                "val/loss": metrics["loss"],
                "val/accuracy": metrics["accuracy"],
                "epoch": epoch,
            }
        )

    run.finish()
    return model

compressed_model = train_compressed(compressed_model, compressed_optimizer, compressed_scheduler, COMPRESSED_EPOCHS)
compressed_metrics = evaluate(compressed_model, eval_dataloader)
print("Compressed (no dendrites) metrics:", compressed_metrics)
print("Compressed (no dendrites) params:", count_parameters(compressed_model))

model.safetensors:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

[Compressed] Epoch 0 | val_loss=0.4861 | val_acc=0.7672
[Compressed] Epoch 1 | val_loss=0.4585 | val_acc=0.7833
[Compressed] Epoch 2 | val_loss=0.4532 | val_acc=0.7913


epoch,▁▅█
step,▁▁▁▁▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇██
train/loss,█████▇▆▆▅▅▄▅▃▄▃▃▃▂▂▂▆▂▆▅▂▂▃▃▄▂▁▁▄▂▃▂▅▂▃▄
val/accuracy,▁▆█
val/loss,█▂▁
epoch,2
step,6300
train/loss,0.44331
val/accuracy,0.79128
val/loss,0.45318


Compressed (no dendrites) metrics: {'loss': 0.45317753793996407, 'accuracy': 0.7912844036697247}
Compressed (no dendrites) params: 4386178


In [16]:
# dentric model
from perforatedai import globals_perforatedai as GPA
from perforatedai import utils_perforatedai as UPA
import torch

GPA.pc.set_unwrapped_modules_confirmed(True)

GPA.pc.set_testing_dendrite_capacity(True)

dendritic_model = AutoModelForSequenceClassification.from_pretrained(
    COMPRESSED_MODEL_NAME,
    num_labels=NUM_LABELS,
)

dendritic_model = UPA.initialize_pai(dendritic_model)
dendritic_model.to(device)

#    Output shape: [batch, seq_len, hidden_dim]; hidden_dim is neuron axis -> 0
output_dim_vec = torch.tensor([-1, -1, 0], device=device)

for block in dendritic_model.bert.encoder.layer:
    # feed-forward blocks
    block.intermediate.dense.set_this_output_dimensions(output_dim_vec)
    block.output.dense.set_this_output_dimensions(output_dim_vec)

    # attention output projection
    block.attention.output.dense.set_this_output_dimensions(output_dim_vec)

    # attention self projections: query, key, value
    block.attention.self.query.set_this_output_dimensions(output_dim_vec)
    block.attention.self.key.set_this_output_dimensions(output_dim_vec)
    block.attention.self.value.set_this_output_dimensions(output_dim_vec)

print("Dendritic model ready. Number of params:", count_parameters(dendritic_model))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


By default skipping base_model. See "Safetensors Errors" section of customization.md to include it.
Running a test of Dendrite Capacity.
Dendritic model ready. Number of params: 4798468


In [17]:
learning_rate = INITIAL_LR

GPA.pai_tracker.set_optimizer(torch.optim.Adam)
GPA.pai_tracker.set_scheduler(torch.optim.lr_scheduler.ReduceLROnPlateau)

optim_args = {'params': dendritic_model.parameters(), 'lr': learning_rate}
sched_args = {'mode': 'max', 'patience': 3}

dend_optimizer, dend_scheduler = GPA.pai_tracker.setup_optimizer(
    dendritic_model,
    optim_args,
    sched_args,
)

print("Dendritic optimizer and scheduler initialized.")

Dendritic optimizer and scheduler initialized.


In [18]:
# def train_with_dendrites(
#     model,
#     optimizer,
#     optim_args,
#     sched_args,
#     max_epochs=20,
#     run_name="compressed-with-dendrites"
# ):
#     run = wandb.init(project=WANDB_PROJECT, name=run_name)
#     epoch = 0
#     global_step = 0

#     while True:
#         print(f"[Dendrites] Starting epoch {epoch}")
#         model.train()

#         for batch in train_dataloader:
#             batch = {k: v.to(device) for k, v in batch.items()}
#             outputs = model(**batch)
#             loss = outputs.loss

#             loss.backward()
#             torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
#             optimizer.step()
#             optimizer.zero_grad()

#             global_step += 1
#             if global_step % 50 == 0:
#                 wandb.log({"train/loss": loss.item(), "step": global_step})

#         # End of epoch -> evaluate
#         val_metrics = evaluate(model, eval_dataloader)
#         val_loss = val_metrics["loss"]
#         val_acc = val_metrics["accuracy"]

#         print(f"[Dendrites] Epoch {epoch} | val_loss={val_loss:.4f} | val_acc={val_acc:.4f}")
#         wandb.log(
#             {
#                 "val/loss": val_loss,
#                 "val/accuracy": val_acc,
#                 "epoch": epoch,
#             }
#         )

#         GPA.pai_tracker.add_extra_score(val_acc, "Val Accuracy")

#         model, restructured, training_complete = GPA.pai_tracker.add_validation_score(
#             val_acc,
#             model,
#         )

#         model.to(device)

#         if training_complete:
#             print("PerforatedAI reports training is complete. Stopping.")
#             break

#         if restructured:
#             print("Model structure changed (dendrites added/absorbed). Rebuilding optimizer & scheduler.")
#             optimizer, _ = GPA.pai_tracker.setup_optimizer(
#                 model,
#                 optim_args,
#                 sched_args,
#             )

#         epoch += 1
#         if epoch >= max_epochs:
#             print("Hit safety max_epochs limit. Stopping.")
#             break

#     run.finish()
#     return model

# dendritic_model = train_with_dendrites(
#     dendritic_model,
#     dend_optimizer,
#     optim_args,
#     sched_args,
#     max_epochs=20,
# )

# dend_metrics = evaluate(dendritic_model, eval_dataloader)
# print("Compressed + dendrites metrics:", dend_metrics)
# print("Compressed + dendrites params:", count_parameters(dendritic_model))

In [19]:
from perforatedai import globals_perforatedai as GPA
from perforatedai import utils_perforatedai as UPA
import torch

GPA.pc.set_unwrapped_modules_confirmed(True)

GPA.pc.set_testing_dendrite_capacity(False)

dendritic_model = AutoModelForSequenceClassification.from_pretrained(
    COMPRESSED_MODEL_NAME,
    num_labels=NUM_LABELS,
)

dendritic_model = UPA.initialize_pai(dendritic_model)
dendritic_model.to(device)

#    Output shape: [batch, seq_len, hidden_dim]; hidden_dim is neuron axis -> 0
output_dim_vec = torch.tensor([-1, -1, 0], device=device)

for block in dendritic_model.bert.encoder.layer:
    # feed-forward blocks
    block.intermediate.dense.set_this_output_dimensions(output_dim_vec)
    block.output.dense.set_this_output_dimensions(output_dim_vec)

    # attention output projection
    block.attention.output.dense.set_this_output_dimensions(output_dim_vec)

    # attention self projections: query, key, value
    block.attention.self.query.set_this_output_dimensions(output_dim_vec)
    block.attention.self.key.set_this_output_dimensions(output_dim_vec)
    block.attention.self.value.set_this_output_dimensions(output_dim_vec)

print("Dendritic model ready. Number of params:", count_parameters(dendritic_model))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


By default skipping base_model. See "Safetensors Errors" section of customization.md to include it.
Running Dendrite Experiment
Dendritic model ready. Number of params: 4798468


In [21]:
def train_with_dendrites(
    model,
    optimizer,
    optim_args,
    sched_args,
    max_epochs=20,
    run_name="compressed-with-dendrites"
):
    run = wandb.init(project=WANDB_PROJECT, name=run_name)
    epoch = 0
    global_step = 0

    while True:
        print(f"[Dendrites] Starting epoch {epoch}")
        model.train()

        for batch in train_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            optimizer.zero_grad()

            global_step += 1
            if global_step % 50 == 0:
                wandb.log({"train/loss": loss.item(), "step": global_step})


        train_metrics = evaluate(model, train_dataloader)
        train_acc = train_metrics["accuracy"]
        train_loss = train_metrics.get("loss", None)

        log_dict = {
            "epoch": epoch,
            "train/accuracy": train_acc,
        }
        if train_loss is not None:
            log_dict["train/epoch_loss"] = train_loss

        wandb.log(log_dict)

        # ----------------------------
        # Evaluate on validation set (orange line in the "good graph")
        # ----------------------------
        val_metrics = evaluate(model, eval_dataloader)
        val_loss = val_metrics["loss"]
        val_acc = val_metrics["accuracy"]

        print(f"[Dendrites] Epoch {epoch} | train_acc={train_acc:.4f} | val_loss={val_loss:.4f} | val_acc={val_acc:.4f}")

        wandb.log(
            {
                "val/loss": val_loss,
                "val/accuracy": val_acc,
                "epoch": epoch,
            }
        )

        # Optional extra score to PAI tracker (not required, but fine)
        GPA.pai_tracker.add_extra_score(val_acc, "Val Accuracy")

        model, restructured, training_complete = GPA.pai_tracker.add_validation_score(
            val_acc,
            model,
        )

        model.to(device)

        wandb.log({
            "pai/restructured": int(restructured),
            "pai/training_complete": int(training_complete),
            "pai/params": sum(p.numel() for p in model.parameters()),
            "epoch": epoch,
        })

        if training_complete:
            print("PerforatedAI reports training is complete. Stopping.")
            break

        if restructured:
            print("Model structure changed (dendrites added/absorbed). Rebuilding optimizer & scheduler.")
            optimizer, _ = GPA.pai_tracker.setup_optimizer(
                model,
                optim_args,
                sched_args,
            )

        epoch += 1
        if epoch >= max_epochs:
            print("Hit safety max_epochs limit. Stopping.")
            break

    run.finish()
    return model


dendritic_model = train_with_dendrites(
    dendritic_model,
    dend_optimizer,
    optim_args,
    sched_args,
    max_epochs=20,
)

# === FINAL EVAL ===
dend_metrics = evaluate(dendritic_model, eval_dataloader)
print("Compressed + dendrites metrics:", dend_metrics)
print("Compressed + dendrites params:", count_parameters(dendritic_model))

[Dendrites] Starting epoch 0
[Dendrites] Epoch 0 | train_acc=0.8533 | val_loss=0.4590 | val_acc=0.7856
Adding validation score 0.78555046
Checking PAI switch with mode n, switch mode DOING_SWITCH_EVERY_TIME, epoch 0, last improved epoch 0, total epochs 0, n: 10, num_cycles: 0
Returning True - switching every time
Importing best Model for switch to PA...
Switching back to N...
Resetting committed to initial rate to False
Saving model before starting normal training to retain PBNodes regardless of next N Phase results
Model structure changed (dendrites added/absorbed). Rebuilding optimizer & scheduler.
[Dendrites] Starting epoch 1
[Dendrites] Epoch 1 | train_acc=0.8892 | val_loss=0.4348 | val_acc=0.8039
Adding validation score 0.80389908
Checking PAI switch with mode n, switch mode DOING_SWITCH_EVERY_TIME, epoch 1, last improved epoch 1, total epochs 1, n: 10, num_cycles: 2
Returning True - switching every time
Importing best Model for switch to PA...
Switching back to N...
Resetting com

KeyboardInterrupt: 

In [ ]:
print("\n=== Final Comparison ===")

print("Baseline:")
print("  Accuracy:", baseline_metrics["accuracy"])
print("  Params:", count_parameters(baseline_model))
print("  Batch time (10 batches):", benchmark_inference(baseline_model, eval_dataloader))

print("\nCompressed (no dendrites):")
print("  Accuracy:", compressed_metrics["accuracy"])
print("  Params:", count_parameters(compressed_model))
print("  Batch time (10 batches):", benchmark_inference(compressed_model, eval_dataloader))

print("\nCompressed + dendrites:")
print("  Accuracy:", dend_metrics["accuracy"])
print("  Params:", count_parameters(dendritic_model))
print("  Batch time (10 batches):", benchmark_inference(dendritic_model, eval_dataloader))

## CIFAR-10 dataset start with new basline and dentrics

In [22]:
!pip install -q torch torchvision tqdm wandb

import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from tqdm.auto import tqdm
import wandb

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [23]:
# CIFAR-10 normalization (standard)
mean = (0.4914, 0.4822, 0.4465)             # per channel mean, normalisation constants
std  = (0.2470, 0.2435, 0.2616)

train_tfms = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

test_tfms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

train_ds = datasets.CIFAR10(root="./data", train=True, download=True, transform=train_tfms)
test_ds  = datasets.CIFAR10(root="./data", train=False, download=True, transform=test_tfms)

BATCH_SIZE = 128
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

NUM_CLASSES = 10

100%|██████████| 170M/170M [00:08<00:00, 20.6MB/s]


In [24]:
def build_resnet18_cifar10(num_classes=10):         # ResNet18 set-up
    m = models.resnet18(weights=None)
    m.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    m.maxpool = nn.Identity()
    m.fc = nn.Linear(m.fc.in_features, num_classes)
    return m

baseline_model = build_resnet18_cifar10(NUM_CLASSES).to(device)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Baseline params:", count_parameters(baseline_model))

Baseline params: 11173962


In [25]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

@torch.no_grad()
def evaluate_accuracy(model, loader):
    model.eval()
    correct, total = 0, 0
    for x, y in loader:
        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)
        logits = model(x)
        preds = logits.argmax(dim=1)
        correct += (preds == y).sum().item()
        total += y.size(0)
    return correct / total

def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    for x, y in loader:
        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * y.size(0)

    return running_loss / len(loader.dataset)

@torch.no_grad()
def benchmark_latency(model, loader, n_batches=30, warmup=10):
    model.eval()
    it = iter(loader)

    # warmup
    for _ in range(warmup):
        x, _ = next(it)
        x = x.to(device, non_blocking=True)
        _ = model(x)
    if device.type == "cuda":
        torch.cuda.synchronize()

    start = time.time()
    for _ in range(n_batches):
        x, _ = next(it)
        x = x.to(device, non_blocking=True)
        _ = model(x)
    if device.type == "cuda":
        torch.cuda.synchronize()
    end = time.time()

    return (end - start) / n_batches

In [26]:
def build_resnet18_cifar10(num_classes=10):
    m = models.resnet18(weights=None)
    m.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    m.maxpool = nn.Identity()
    m.fc = nn.Linear(m.fc.in_features, num_classes)
    return m

def build_shufflenetv2_cifar10(num_classes=10):
    m = models.shufflenet_v2_x1_0(weights=None)
    # replace classifier
    m.fc = nn.Linear(m.fc.in_features, num_classes)
    return m

In [28]:
import copy


WANDB_PROJECT = "perforatedai-hackathon-cifar10"

hist_A = {"epoch": [], "train_acc": [], "test_acc": []}

model_A = build_resnet18_cifar10(NUM_CLASSES).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_A.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)

run = wandb.init(project=WANDB_PROJECT, name="A_resnet18_no_dendrites", reinit=True)

EPOCHS_A = 20
best_acc = 0.0
best_state = None

for epoch in range(EPOCHS_A):
    train_loss = train_one_epoch(model_A, train_loader, optimizer, criterion)
    train_acc = evaluate_accuracy(model_A, train_loader)
    test_acc  = evaluate_accuracy(model_A, test_loader)
    scheduler.step()

    hist_A["epoch"].append(epoch)
    hist_A["train_acc"].append(train_acc)
    hist_A["test_acc"].append(test_acc)

    wandb.log({
        "epoch": epoch,
        "train/loss": train_loss,
        "train/accuracy": train_acc,
        "test/accuracy": test_acc,
        "lr": optimizer.param_groups[0]["lr"],
    })

    print(f"[A] epoch={epoch:02d} train_acc={train_acc:.4f} test_acc={test_acc:.4f}")

    if test_acc > best_acc:
        best_acc = test_acc
        best_state = copy.deepcopy(model_A.state_dict())

model_A.load_state_dict(best_state)
final_A_acc = evaluate_accuracy(model_A, test_loader)
final_A_params = count_parameters(model_A)
final_A_latency = benchmark_latency(model_A, test_loader)

wandb.log({"final/accuracy": final_A_acc, "final/params": final_A_params, "final/latency_sec_per_batch": final_A_latency})
run.finish()

print("\n=== A FINAL ===")
print("Accuracy:", final_A_acc)
print("Params:", final_A_params)
print("Latency (sec/batch):", final_A_latency)

epoch,▁
lr,▁
test/accuracy,▁
train/accuracy,▁
train/loss,▁
epoch,0
lr,0.09938
test/accuracy,0.3797
train/accuracy,0.37218
train/loss,2.09956


[A] epoch=00 train_acc=0.3962 test_acc=0.4069
[A] epoch=01 train_acc=0.5121 test_acc=0.5097
[A] epoch=02 train_acc=0.6385 test_acc=0.6416
[A] epoch=03 train_acc=0.5767 test_acc=0.5860
[A] epoch=04 train_acc=0.7452 test_acc=0.7369
[A] epoch=05 train_acc=0.7863 test_acc=0.7835
[A] epoch=06 train_acc=0.8126 test_acc=0.8041
[A] epoch=07 train_acc=0.7668 test_acc=0.7583
[A] epoch=08 train_acc=0.7981 test_acc=0.7837
[A] epoch=09 train_acc=0.8522 test_acc=0.8350
[A] epoch=10 train_acc=0.8757 test_acc=0.8525
[A] epoch=11 train_acc=0.8593 test_acc=0.8395
[A] epoch=12 train_acc=0.8803 test_acc=0.8509
[A] epoch=13 train_acc=0.9167 test_acc=0.8798
[A] epoch=14 train_acc=0.9391 test_acc=0.8966
[A] epoch=15 train_acc=0.9470 test_acc=0.8987
[A] epoch=16 train_acc=0.9614 test_acc=0.9080
[A] epoch=17 train_acc=0.9735 test_acc=0.9175
[A] epoch=18 train_acc=0.9799 test_acc=0.9233
[A] epoch=19 train_acc=0.9820 test_acc=0.9215


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
final/accuracy,▁
final/latency_sec_per_batch,▁
final/params,▁
lr,███▇▇▇▆▆▅▅▄▃▃▂▂▂▁▁▁▁
test/accuracy,▁▂▄▃▅▆▆▆▆▇▇▇▇▇██████
train/accuracy,▁▂▄▃▅▆▆▅▆▆▇▇▇▇▇█████
train/loss,█▆▅▄▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁
epoch,19
final/accuracy,0.9233
final/latency_sec_per_batch,0.01588



=== A FINAL ===
Accuracy: 0.9233
Params: 11173962
Latency (sec/batch): 0.015875013669331868


In [29]:
hist_B = {"epoch": [], "train_acc": [], "test_acc": []}

model_B = build_shufflenetv2_cifar10(NUM_CLASSES).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_B.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)

run = wandb.init(project=WANDB_PROJECT, name="B_shufflenet_no_dendrites", reinit=True)

EPOCHS_B = 20
best_acc = 0.0
best_state = None

for epoch in range(EPOCHS_B):
    train_loss = train_one_epoch(model_B, train_loader, optimizer, criterion)
    train_acc = evaluate_accuracy(model_B, train_loader)
    test_acc  = evaluate_accuracy(model_B, test_loader)
    scheduler.step()

    hist_B["epoch"].append(epoch)
    hist_B["train_acc"].append(train_acc)
    hist_B["test_acc"].append(test_acc)

    wandb.log({
        "epoch": epoch,
        "train/loss": train_loss,
        "train/accuracy": train_acc,
        "test/accuracy": test_acc,
        "lr": optimizer.param_groups[0]["lr"],
    })

    print(f"[B] epoch={epoch:02d} train_acc={train_acc:.4f} test_acc={test_acc:.4f}")

    if test_acc > best_acc:
        best_acc = test_acc
        best_state = copy.deepcopy(model_B.state_dict())

model_B.load_state_dict(best_state)
final_B_acc = evaluate_accuracy(model_B, test_loader)
final_B_params = count_parameters(model_B)
final_B_latency = benchmark_latency(model_B, test_loader)

wandb.log({"final/accuracy": final_B_acc, "final/params": final_B_params, "final/latency_sec_per_batch": final_B_latency})
run.finish()

print("\n=== B FINAL ===")
print("Accuracy:", final_B_acc)
print("Params:", final_B_params)
print("Latency (sec/batch):", final_B_latency)

[B] epoch=00 train_acc=0.3045 test_acc=0.3267
[B] epoch=01 train_acc=0.3970 test_acc=0.4288
[B] epoch=02 train_acc=0.4321 test_acc=0.4554
[B] epoch=03 train_acc=0.4182 test_acc=0.4480
[B] epoch=04 train_acc=0.4749 test_acc=0.5011
[B] epoch=05 train_acc=0.4758 test_acc=0.5035
[B] epoch=06 train_acc=0.4921 test_acc=0.5243
[B] epoch=07 train_acc=0.5392 test_acc=0.5668
[B] epoch=08 train_acc=0.5277 test_acc=0.5417
[B] epoch=09 train_acc=0.5487 test_acc=0.5711
[B] epoch=10 train_acc=0.5684 test_acc=0.5956
[B] epoch=11 train_acc=0.5881 test_acc=0.6055
[B] epoch=12 train_acc=0.5993 test_acc=0.6215
[B] epoch=13 train_acc=0.6103 test_acc=0.6249
[B] epoch=14 train_acc=0.6391 test_acc=0.6443
[B] epoch=15 train_acc=0.6513 test_acc=0.6537
[B] epoch=16 train_acc=0.6669 test_acc=0.6651
[B] epoch=17 train_acc=0.6788 test_acc=0.6739
[B] epoch=18 train_acc=0.6914 test_acc=0.6864
[B] epoch=19 train_acc=0.6933 test_acc=0.6895


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
final/accuracy,▁
final/latency_sec_per_batch,▁
final/params,▁
lr,███▇▇▇▆▆▅▅▄▃▃▂▂▂▁▁▁▁
test/accuracy,▁▃▃▃▄▄▅▆▅▆▆▆▇▇▇▇████
train/accuracy,▁▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇████
train/loss,█▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁▁
epoch,19
final/accuracy,0.6895
final/latency_sec_per_batch,0.01707



=== B FINAL ===
Accuracy: 0.6895
Params: 1263854
Latency (sec/batch): 0.017073949178059895


In [30]:
model_C = build_shufflenetv2_cifar10(NUM_CLASSES)
model_C = UPA.initialize_pai(model_C)  # must be before .to(device)
model_C = model_C.to(device)

print("C params (initial):", count_parameters(model_C))

Running Dendrite Experiment
C params (initial): 2511528


In [ ]:
learning_rate = 0.05

GPA.pai_tracker.set_optimizer(torch.optim.SGD)
GPA.pai_tracker.set_scheduler(torch.optim.lr_scheduler.CosineAnnealingLR)

optim_args = {"params": model_C.parameters(), "lr": learning_rate, "momentum": 0.9, "weight_decay": 5e-4}
sched_args = {"T_max": 20}

optimizer_C, scheduler_C = GPA.pai_tracker.setup_optimizer(model_C, optim_args, sched_args)

criterion = nn.CrossEntropyLoss()

In [ ]:
hist_C = {"epoch": [], "train_acc": [], "test_acc": [], "restructured_epoch": []}

def train_with_dendrites_cifar(
    model,
    optimizer,
    optim_args,
    sched_args,
    max_epochs=40,
    run_name="C_shufflenet_dendrites"
):
    run = wandb.init(project=WANDB_PROJECT, name=run_name, reinit=True)

    epoch = 0
    while True:
        # ---- Train epoch ----
        model.train()
        for x, y in train_loader:
            x = x.to(device, non_blocking=True)
            y = y.to(device, non_blocking=True)

            optimizer.zero_grad(set_to_none=True)
            logits = model(x)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()

        # ---- Epoch metrics ----
        train_acc = evaluate_accuracy(model, train_loader)
        test_acc  = evaluate_accuracy(model, test_loader)

        # ---- PAI decision based on score (maximize accuracy) ----
        model, restructured, training_complete = GPA.pai_tracker.add_validation_score(
            test_acc, model
        )
        model = model.to(device)

        # If structure changed, rebuild optimizer/scheduler via tracker
        if restructured:
            hist_C["restructured_epoch"].append(epoch)
            optimizer, _ = GPA.pai_tracker.setup_optimizer(model, optim_args, sched_args)

        # Save history
        hist_C["epoch"].append(epoch)
        hist_C["train_acc"].append(train_acc)
        hist_C["test_acc"].append(test_acc)

        # Log to wandb
        wandb.log({
            "epoch": epoch,
            "train/accuracy": train_acc,
            "test/accuracy": test_acc,
            "pai/restructured": int(restructured),
            "pai/training_complete": int(training_complete),
            "pai/params": count_parameters(model),
        })

        print(f"[C] epoch={epoch:02d} train_acc={train_acc:.4f} test_acc={test_acc:.4f} restructured={restructured}")

        if training_complete:
            print("PerforatedAI reports training complete.")
            break

        epoch += 1
        if epoch >= max_epochs:
            print("Hit safety max_epochs limit.")
            break

    run.finish()
    return model

model_C = train_with_dendrites_cifar(
    model_C, optimizer_C, optim_args, sched_args, max_epochs=40
)

final_C_acc = evaluate_accuracy(model_C, test_loader)
final_C_params = count_parameters(model_C)
final_C_latency = benchmark_latency(model_C, test_loader)

print("\n=== C FINAL ===")
print("Accuracy:", final_C_acc)
print("Params:", final_C_params)
print("Latency (sec/batch):", final_C_latency)

Adding validation score 0.35680000
Checking PAI switch with mode n, switch mode DOING_SWITCH_EVERY_TIME, epoch 0, last improved epoch 0, total epochs 0, n: 10, num_cycles: 0
Returning True - switching every time
Importing best Model for switch to PA...
Switching back to N...
Resetting committed to initial rate to False
Saving model before starting normal training to retain PBNodes regardless of next N Phase results
[C] epoch=00 train_acc=0.3480 test_acc=0.3568 restructured=1
Adding validation score 0.48500000
Checking PAI switch with mode n, switch mode DOING_SWITCH_EVERY_TIME, epoch 1, last improved epoch 1, total epochs 1, n: 10, num_cycles: 2
Returning True - switching every time
Importing best Model for switch to PA...
Switching back to N...
Resetting committed to initial rate to False
Saving model before starting normal training to retain PBNodes regardless of next N Phase results
[C] epoch=01 train_acc=0.4566 test_acc=0.4850 restructured=1
Adding validation score 0.51860000
Check

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(10,6))

# Dendritic run (green/orange)
plt.plot(hist_C["epoch"], [a*100 for a in hist_C["train_acc"]], label="Train (with dendrites)")
plt.plot(hist_C["epoch"], [a*100 for a in hist_C["test_acc"]],  label="Val/Test (with dendrites)")

# No-dendrites ShuffleNet baseline (red/blue) = B
plt.plot(hist_B["epoch"], [a*100 for a in hist_B["train_acc"]], label="Train (no dendrites)", linestyle="--")
plt.plot(hist_B["epoch"], [a*100 for a in hist_B["test_acc"]],  label="Val/Test (no dendrites)", linestyle="--")

# Vertical lines where dendrites were added
for e in hist_C["restructured_epoch"]:
    plt.axvline(x=e, linestyle="--")

plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.title("What a Good Graph Should Look Like (CIFAR-10)")
plt.legend()
plt.grid(True)
plt.show()

# Comparison bars (A vs B vs C)
labels = ["A ResNet-18", "B ShuffleNet", "C ShuffleNet+dendrites"]
accs   = [final_A_acc*100, final_B_acc*100, final_C_acc*100]
params = [final_A_params, final_B_params, final_C_params]
lats   = [final_A_latency, final_B_latency, final_C_latency]

plt.figure(figsize=(10,4))
plt.bar(labels, accs)
plt.ylabel("Accuracy (%)")
plt.title("Final Accuracy Comparison")
plt.grid(axis="y")
plt.show()

plt.figure(figsize=(10,4))
plt.bar(labels, params)
plt.ylabel("Trainable Params")
plt.title("Parameter Count Comparison")
plt.grid(axis="y")
plt.show()

plt.figure(figsize=(10,4))
plt.bar(labels, lats)
plt.ylabel("Seconds per Batch")
plt.title("Latency Comparison")
plt.grid(axis="y")
plt.show()

print("\n=== FINAL SUMMARY ===")
print("A ResNet-18:", final_A_acc, final_A_params, final_A_latency)
print("B ShuffleNet:", final_B_acc, final_B_params, final_B_latency)
print("C ShuffleNet+dendrites:", final_C_acc, final_C_params, final_C_latency)
print("Dendrite events (epochs):", hist_C["restructured_epoch"])